# 비화재보 분류 


---
## 1. 데이터 전처리     
---

### 각 년도별 데이터 결합 

In [3]:
import pandas as pd
import numpy as np 
import re
import pickle
import xlrd
start_year= 2016
end_year =2021

In [2]:

df_all =pd.DataFrame()
#년도합치기
for year in range(start_year,end_year+1):
#시트 이름 가져오기    
    wb = xlrd.open_workbook(f"{year}년 전체 신고건수_관할 및 재난주소 추가.xls")
    globals()[f'df_{year}']=pd.DataFrame()
#Sheet 합치기
    for  i in range(len(wb.sheets())):
        tmp = pd.read_excel(f"{year}년 전체 신고건수_관할 및 재난주소 추가.xls",sheet_name = wb.sheet_names()[i])['신고내용'].dropna(axis=0).T
        globals()[f'df_{year}']= pd.concat([globals()[f'df_{year}'],tmp], axis = 0)
    df_all= pd.concat([df_all,globals()[f'df_{year}']], axis = 0)


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but 

In [13]:
# with open(f"119_신고내용_{start_year} - {end_year}.pickle","wb") as fw:
#     pickle.dump(df_all, fw)   


In [4]:
with open(f"119_신고내용_{start_year} - {end_year}.pickle",'rb') as fr:
    df_all = pickle.load(fr)

### 중복 문자열 제거

In [5]:
print(len(df_all))
df_all= pd.DataFrame(set(df_all[0]))
print(len(df_all))
print('예시')
print(df_all.head(3))

1711724
1368100
예시
                                                   0
0                     태전동 롯데아파트 1동 508호/심부전증 환자 거동불가
1  동구 화랑로35길 58-5 / 재활용 가게 / 난로때문에 화재가 있었다 /// 연기...
2                                     구암교회 뒷산쪽, 멧돼지 


###  신고내용 전처리 

In [59]:
from konlpy.tag import Okt 
okt = Okt()
print(okt.pos('속보기')) 
#속보기 단어인식을 못함 
#한단어로 인식 X
print(okt.pos('경보가울림'))
# 경 보가  로 판단됨(띄어쓰기를 제대로 하지않았을때)
print(okt.pos('경보기')) 
print(okt.pos('경보')) 
print(okt.pos('속보')) 

[('속', 'Modifier'), ('보기', 'Noun')]
[('경보기', 'Noun')]
[('경보', 'Noun')]
[('속보', 'Noun')]
[('경', 'Modifier'), ('보가', 'Noun'), ('울', 'Modifier'), ('린다', 'Noun')]
[('경보', 'Noun'), ('가', 'Josa'), ('울', 'Modifier'), ('린다', 'Noun')]


In [7]:
df_all2 = df_all.copy()
for i in range(len(df_all2)):
    tmp = re.sub(r'[^가-힣+ ]',' ',df_all2.iloc[i,0]) # 한글말고 제거
    tmp = re.sub('속보기','속보',tmp) # 속보기 ->속보로 단어 치환    
    df_all2.iloc[i,0] = tmp
print(df_all2.head(3))
print(len(df_all2))

                                                   0
0                     태전동 롯데아파트  동    호 심부전증 환자 거동불가
1  동구 화랑로  길        재활용 가게   난로때문에 화재가 있었다     연기...
2                                     구암교회 뒷산쪽  멧돼지 
1368100


In [8]:
#속보기 단어 제거 확인
print(f'속보기 단어 수   : {df_all[0].apply(lambda x: re.search("속보기", x)is not None).sum()}')
print(f'속보기 단어 제거 :{df_all2[0].apply(lambda x: re.search("속보기", x)is not None).sum()}')
print(f'제거 안된 문장 :{df_all2[0][df_all2[0].apply(lambda x: re.search("속보기", x)is not None)]}')
### 왜 1개 제거가 안됏는지?

속보기 단어 수   : 7937
속보기 단어 제거 :1
제거 안된 문장 :702307    속보기 앞산        추가 오작동 확인 출동취소
Name: 0, dtype: object


In [9]:
# i = 575070 
# tmp = re.sub(r'[^가-힣+ ]',' ',df_all2.iloc[i,0]) # 한글말고 제거
# tmp = re.sub('속보기','속보',tmp)
# print(tmp)
    

###  문장 -> 형태소 형태로 변환
형태소 분석(Noun, Adjective) -명사와 형용사만  
한글자를 배제  
불용어 제외  

In [10]:

sents = []
for i in range(len(df_all2[0])):
    print("\r[tokenizing]  {}  /  {}   {}  %".format(i+1, len(df_all2[0]), round( (i / len(df_all2[0])) * 100 , 1 ) ), end = '    ', flush = False)
    pos_res = okt.pos(df_all2[0].iloc[i])
    
    STOP_WORDS = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    keywords = []
    for word, pos in pos_res:
        if ((pos == "Noun")|(pos == "Adjective"))&(len(word) >= 2)&(word not in STOP_WORDS):
            keywords.append(word)
    sents.append(' '.join(keywords))
print(sents[0:5])

[tokenizing]  1368100  /  1368100   100.0  %    ['태전동 롯데 아파트 심부전증 환자 거동 불가', '동구 화랑로 재활용 가게 난로 때문 화재 있었다 연기 진화 완료 현장 지휘 구급차 요청 사후 조사', '교회 뒷산 멧돼지', '달성 공원 대신 여인숙 할머니 온몸 통증 호소', '리치 여성 우울증 본동 주영 본동']                                                                                                                                                                                                                                                                                                                                                          


###  전처리 전후 키워드 갯수 확인
비화재보 주요 키워드 : 경보, 속보  
비화재보 제외 키워드 : 테스트, 오인주의, 훈련, 점검   

#### 경보, 속보  

In [11]:
check = []
for i in range(len(sents)) :
    if '속보' in sents[i]:
            check.append(sents[i])
print(f'기존 [속보] 단어 수: {df_all2[0].apply(lambda x: re.search("속보", x)is not None).sum()}')
print(f'전처리 후 [속보] 단어 수: {len(check)}')  


기존 [속보] 단어 수: 16048
전처리 후 [속보] 단어 수: 16038


In [12]:
check = []
for i in range(len(sents)) :
    if '경보' in sents[i]:
            check.append(sents[i])
print(f'기존 [경보] 단어 수: {df_all2[0].apply(lambda x: re.search("경보", x)is not None).sum()}')
print(f'전처리 후 [경보] 단어 수: {len(check)}')  


기존 [경보] 단어 수: 16068
전처리 후 [경보] 단어 수: 15951


#### 테스트, 오인주의, 훈련, 점검   

In [13]:
check = []
for i in range(len(sents)) :
    if '점검' in sents[i]:
            check.append(sents[i])
print(f'기존 [점검] 단어 수: {df_all2[0].apply(lambda x: re.search("점검", x)is not None).sum()}')
print(f'전처리 후 [점검] 단어 수: {len(check)}')  

기존 [점검] 단어 수: 1950
전처리 후 [점검] 단어 수: 1948


In [14]:
check = []
for i in range(len(sents)) :
    if '훈련' in sents[i]:
            check.append(sents[i])
print(f'기존 [훈련] 단어 수: {df_all2[0].apply(lambda x: re.search("훈련", x)is not None).sum()}')
print(f'전처리 후 [훈련] 단어 수: {len(check)}')  

기존 [훈련] 단어 수: 3099
전처리 후 [훈련] 단어 수: 3099


In [15]:
check = []
for i in range(len(sents)) :
    if '오인' in sents[i]:
            check.append(sents[i])
print(f'기존 [오인] 단어 수: {df_all2[0].apply(lambda x: re.search("오인", x)is not None).sum()}')
print(f'전처리 후 [오인] 단어 수: {len(check)}')  

기존 [오인] 단어 수: 12399
전처리 후 [오인] 단어 수: 12188


In [16]:
check = []
for i in range(len(sents)) :
    if '테스트' in sents[i]:
            check.append(sents[i])
print(f'기존 [테스트] 단어 수: {df_all2[0].apply(lambda x: re.search("테스트", x)is not None).sum()}')
print(f'전처리 후 [테스트] 단어 수: {len(check)}')  

기존 [테스트] 단어 수: 10045
전처리 후 [테스트] 단어 수: 10045


In [21]:
with open(f"119전처리_{start_year}-{end_year}.pickle","wb") as fw:
    pickle.dump(sents, fw)   

In [22]:
with open(f"119전처리_{start_year}-{end_year}.pickle",'rb') as fr:
    sents = pickle.load(fr)
# df_sents = pd.DataFrame(sents)

---
## 2. 분석
---

### CountVectorizer Vectorizing


In [32]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
cv_sents=vect.fit_transform(sents)
print(cv_sents.shape)
# cv_data=cv_sents.toarray()
# cv_data

(1368100, 69363)


### TF-IDF Vectorizing


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
tfidf_sents =vectorizer.fit_transform(sents)
print(tfidf_sents.shape)

(1368100, 69363)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(min_df=0.0, analyzer='char', sublinear_tf=True, ngram_range=(1,3), max_features=10000)
tfidf_sents =vectorizer.fit_transform(df_sents[0])
print(tfidf_sents.shape)
# 파라미터에 값입력

(778936, 10000)


### Word2vec(CBOW)


In [38]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []
for review in sents:
    sentences.append(review.split())

In [39]:
# 파라미터
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3 # 0.001

In [ ]:
from gensim.models import word2vec
print("Training model ....")
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count, window=context, sample=downsampling)

In [67]:
tmp1 = model.wv.most_similar("속보")
tmp2= model.wv.most_similar("경보")
print(f'속보\n: {tmp1} \n')
print(f'경보\n: {tmp2} \n')

속보
: [('설비', 0.5565919876098633), ('자동', 0.5560749769210815), ('경보', 0.5363364815711975), ('소방설비', 0.5336592197418213), ('작동', 0.4983700215816498), ('인과', 0.49686312675476074), ('소방시설', 0.4892948269844055), ('점검', 0.48302412033081055), ('경보기', 0.47331950068473816), ('테스트', 0.4692937731742859)] 

경보
: [('화재경보기', 0.7432859539985657), ('보가', 0.7132909297943115), ('경보기', 0.7123016715049744), ('경종', 0.6195616722106934), ('감지기', 0.6176596879959106), ('수신기', 0.603028416633606), ('알람', 0.5819022059440613), ('번쩍', 0.5372390151023865), ('속보', 0.5363364815711975), ('발신기', 0.534061074256897)] 



In [41]:
def get_features(words,model, num_features):
    feature_vector = np.zeros((num_features), dtype=np.float32)
    num_words = 0
    index2word_set = set(model.wv.index2word)
    for w in words:
        if w in index2word_set:
            num_words +=1
            feature_vector = np.add(feature_vector, model[w])
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [42]:
def get_dataset(sentence, model, num_features):
    dataset = list()
    for s in sentence:
        dataset.append(get_features(s, model, num_features))
    FeatureVecs = np.stack(dataset)
    return FeatureVecs

In [43]:
train_data_vecs = get_dataset(sentences, model, num_features)
print(train_data_vecs.shape)

C:\Users\user\AppData\Local\Temp/ipykernel_2544/2435305589.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  feature_vector = np.add(feature_vector, model[w])
C:\Users\user\AppData\Local\Temp/ipykernel_2544/2435305589.py:9: RuntimeWarning: invalid value encountered in true_divide
  feature_vector = np.divide(feature_vector, num_words)


(1368100, 300)


In [44]:
print(train_data_vecs.shape)

(1368100, 300)


In [70]:
train_data_vecs

array([[-0.17214534, -0.4194798 , -0.06259897, ..., -0.3146527 ,
        -0.31471124,  0.0989478 ],
       [ 0.41353115,  0.58591795,  0.31289923, ..., -0.5308654 ,
        -0.11253642, -0.37847427],
       [ 0.5521837 , -0.76813   , -0.1612042 , ..., -0.69882536,
        -0.11171618,  0.4713359 ],
       ...,
       [-0.6271333 , -0.33706298, -1.0009954 , ...,  0.6093799 ,
        -0.6604633 ,  0.558411  ],
       [-0.16968767,  0.04514722,  0.14016913, ...,  0.25245786,
        -0.5659022 ,  0.27416438],
       [-1.0406647 ,  0.03921632,  1.2199225 , ...,  0.13699155,
         0.25819275,  0.86349905]], dtype=float32)